# The Kuramoto-Sivashinsky equation

The Kuramoto-Sivashinsky (KS) equation is a non-linear partial differential equation with one spatial and one temporal dimension. The *integral form* reads

$$u_t + u_{xx} + u_{xxxx} + \frac{1}{2} u_x^2 = 0,$$

where we assume a finite spatial domain $x \in [0, L]$ with periodic boundary conditions and a positive time parameter $t \in [0, \infty)$.
Another form, the *derivative form*, can be obtained by differentiating with respect to $x$ and substituting $v = u_x$

$$v_t + v_{xx} + v_{xxxx} + v v_x = 0 .$$

## Numerical solution: derivative form

First, we recast the KS equation in derivative form to

$$v_t = L(v) + N(v),$$

defining the linear part $L(v) = -v_{xx} - v_{xxxx}$ and non-linear part $N(v) = -v v_x$.
While the time variable is discretized in the real domain $v^n(x) = v(x, n \Delta t)$ along small time-steps $\Delta t \ll 1$, the spatial variable is discretized via a discrete Fourier transform

$$\tilde{v}^n(k) = \sum_{m=0}^{N-1} v^n(m \Delta x) e^{-i \frac{2\pi k}{L} m\Delta x}, \quad
v^n(m \Delta x) = \frac{1}{N} \sum_{k=0}^{N-1} \tilde{v}^n(k) e^{i \frac{2\pi k}{L} m\Delta x}$$

on small spatial increments $\Delta x = \frac{L}{N-1} \ll 1$ and the real wave numbers $\alpha_k = \frac{2\pi k}{L}$.
The linear part will be numerically solved using [Crank-Nicolson time-stepping](https://en.wikipedia.org/wiki/Crank%E2%80%93Nicolson_method) and the non-linear terms by the [Adams-Bashforth 2-step method](https://en.wikipedia.org/wiki/Linear_multistep_method#Adams%E2%80%93Bashforth_methods). Together, this is abbreviated by CNAB2. In short, the Crank-Nicolson method integrates some general PDE

$$\frac{\partial y}{\partial t} = F(y, x, t, \frac{\partial y}{\partial x}, \dots) \quad \longrightarrow \quad
\frac{y^{n+1} - y^n}{\Delta t} = \frac{1}{2} [F^{n+1} + F^n],$$

whereas 2-step Adams-Bashforth integrates (non-linear) ODEs

$$\frac{\mathrm{d} y}{\mathrm{d} t} = f(t, y) \quad \longrightarrow \quad
y^{n+1} = y^n + \Delta t \left[\frac{3}{2} f^n - \frac{1}{2} f^{n-1}\right].$$

Applied to the KS equation in derivative form, we obtain

$$\frac{v^{n+1} - v^n}{\Delta t} = \frac{1}{2} L(v^{n+1} + v^n) + \frac{3}{2} N(v^n) - \frac{1}{2} N(v^{n-1}) \\
(1 - \frac{\Delta t}{2} L) v^{n+1} = (1 + \frac{\Delta t}{2} L) v^n + \frac{3\Delta t}{2} N(v^n) - \frac{\Delta t}{2} N(v^{n-1}).$$

Finally, we go to Fourier space so that $v^n \to \bm{v}^n$ now is a vector of Fourier coefficients and

$$L(v) \to \mathrm{diag}(\alpha_k^2 - \alpha_k^4)_{k=0,\dots,N-1} \bm{v}\\
N(v) = -\frac{1}{2} \frac{\mathrm{d}}{\mathrm{d}x} v^2 \to
\frac{i}{2} \mathrm{diag}(\alpha_k)_k \mathcal{F}\{v^2\}$$

become matrices (in particular $L$ is diagonal). What's left to solve is a matrix equation that is just iterated to obtain the (approximate) time evolution of the KS equation:

$$\bm{\tilde{v}}^{n+1} = B \left[ A \bm{\tilde{v}}^n + \frac{3\Delta t}{2} \bm{N}^n - \frac{\Delta t}{2} \bm{N}^{n-1} \right],\quad 
A = I + \frac{\Delta t}{2} L,\quad B = \left[ I - \frac{\Delta t}{2} L \right]^{-1}.$$

## Implementation in Julia

As a first "naive" version of this CNAB2 stepping, which employs no particular performance optimizations, consider the following:

In [ ]:
using FFTW

function ksIntegrateSlow(u, Lx, dt, Nt, nplot)
    Nx = length(u)
    kx = vcat(0:Nx/2-1, 0, -Nx/2+1:-1)  # Integer wavenumbers
    alpha = 2 * pi / Lx * kx
    D = 1im * alpha
    L = alpha .^ 2 - alpha .^ 4
    G = -0.5 * D
    nsave = round(Int64, Nt / nplot) + 1  # Total number of saved time steps

    x = Lx / Nx * collect(0:Nx-1)
    t = dt * nplot * collect(0:nsave-1)
    U = zeros(Float64, nsave, Nx)

    # Convenience variables
    dt2 = dt / 2
    dt32 = 3 * dt / 2
    A = ones(Nx) + dt2 * L
    B = (ones(Nx) - dt2 * L) .^ -1

    Nn = G .* fft(u .* u)
    Nn1 = Nn
    U[1, :] = u  # Save boundary condition
    u = fft(u)
    counter = 2

    for n = 0:Nt-1
        Nn1 = Nn
        Nn = G .* fft(real(ifft(u)) .^ 2)
        u = B .* (A .* u + dt32 * Nn - dt2 * Nn1)

        # Save every nplot-th solution to returned array u
        if mod(n, nplot) == 0
            U[counter, :] = real(ifft(u))
            counter += 1
        end
    end
    U, t, x
end

The improved version utilizes:
- *In-place planned* FFTs: *in-place* means that the FFT modifies the input array which avoids allocating memory; *planned* FFTs are used if several FFTs of the same size are performed so that "FFT plans" can be pre-computed (the FFT operation is then performed as a matrix-vector multiplication `plan*array`)
- `@inbounds` macro: tells the compiler to not check whether array indices are actually in bound
- `@fastmath` macro: floating-point optimizations for real numbers (might lead to differences in the numerical results!)

In [ ]:
function ksIntegrateFast(u0, Lx, dt, Nt, nplot)
    Nx = length(u0)
    kx = vcat(0:Nx/2-1, 0, -Nx/2+1:-1)
    alpha = 2 * pi / Lx * kx
    D = 1im * alpha
    L = alpha .^ 2 - alpha .^ 4
    G = -0.5 * D
    nsave = round(Int64, Nt / nplot) + 1

    x = Lx / Nx * collect(0:Nx-1)
    t = dt * nplot * collect(0:nsave-1)
    U = zeros(Float64, nsave, Nx)
    U[1, :] = u0
    u = Vector{ComplexF64}(u0)  # Complex type necessary for planned FFTs

    # Convenience variables
    dt2 = dt / 2
    dt32 = 3 * dt / 2
    A = ones(Nx) + dt2 * L
    B = (ones(Nx) - dt2 * L) .^ -1

    # Compute in-place planned FFTs
    FFT! = plan_fft!(u, flags=FFTW.ESTIMATE)
    IFFT! = plan_ifft!(u, flags=FFTW.ESTIMATE)

    Nn = G .* fft(u .^ 2)
    Nn1 = copy(Nn)
    FFT! * u
    counter = 2

    for n = 0:Nt-1
        # Compute Nn = G .* fft(real(ifft(u)).^2) inbounds with fastmath
        for i = 1:length(Nn)
            @inbounds Nn1[i] = Nn[i]
            @inbounds Nn[i] = u[i]
        end

        IFFT! * Nn

        for i = 1:length(Nn)
            @fastmath @inbounds Nn[i] = Nn[i] * Nn[i]
        end

        FFT! * Nn

        for i = 1:length(Nn)
            @fastmath @inbounds Nn[i] = G[i] * Nn[i]
        end

        # Compute u = B .* (A .* u + dt32*Nn - dt2*Nn1) inbounds with fastmath
        for i = 1:length(u)
            @fastmath @inbounds u[i] = B[i] * (A[i] * u[i] + dt32 * Nn[i] - dt2 * Nn1[i])
        end

        if mod(n, nplot) == 0
            U[counter, :] = real(ifft(u))
            counter += 1
        end
    end
    U, t, x
end

## Plotting and animation the solution

We will be using the `Makie` library and in particular the `GLMakie` backend (which uses OpenGL to compute plots the GPU) for animated and interactive plots. This also brings nice layout/UI capabilities with it. We can write animations to an .mp4 file (works out of the box!) or actually run the animation live in an unbounded loop.

In [ ]:
using GLMakie
set_theme!(theme_black())

In [ ]:
# run test and plot heatmap
Lx = 256
Nx = 2048
dt = 1 / 32
nplot = 4
Nt = 2048

x = Lx / Nx * collect(1:Nx)
u0 = @. cos(x) + 0.1 * cos(x / 16) * (1 + 2 * sin(x / 16))
@time Uslow, tslow, xslow = ksIntegrateSlow(u0, Lx, dt, Nt, nplot);
@time Ufast, tfast, xfast = ksIntegrateFast(u0, Lx, dt, Nt, nplot);

# Check whether fast and slow integrator produce same results
print("Uslow == Ufast: ", isapprox(Uslow, Ufast))
heatmap(xfast, tfast, Ufast'; axis=(xlabel="x", ylabel="time steps"))

In [ ]:
# animate KS cross section in Makie.jl
time = Observable(1)
y = @lift(Ufast[$time, :])
fig = Figure()
ax = Axis(fig[1, 1]; xlabel="x", ylabel="U(x)",
    limits=(x[1], x[end], -3.0, 3.0))
lines!(ax, x, y; color=:springgreen)

framerate = 30
timestamps = 1:size(Ufast, 1)

record(fig, "cross_section_anim.mp4", timestamps; framerate=framerate) do t
    time[] = t
end

In [ ]:
# Live animating the heatmap
fps = 30
fig = Figure()
ax = Axis(fig[1, 1])
hidedecorations!(ax)

# KS parameters
Lx = 32
Nx = 1024
dt = 1 / 16
Nt = 1600  # Number of time steps within figure
nplot = 8
nsave = round(Int64, Nt / nplot) + 1

x = Lx / Nx * collect(1:Nx)
t = dt * nplot * collect(0:nsave-1)
u0 = cos.(x) .+ 0.1 * cos.(x / 16) .* (1 .+ 2 * sin.(x / 16))
# u0  = rand(Nx)  # Uniform distribution in [0,1]
U = zeros(Float64, nsave, Nx)
U[1, :] = u0
u = fft(u0)

# CNAB2 variables
kx = vcat(0:Nx/2-1, 0, -Nx/2+1:-1)
# kx = collect(0:Nx-1)  # Integer wavenumbers in stable ordering?
alpha = 2 * pi / Lx * kx
D = 1im * alpha
L = alpha .^ 2 - alpha .^ 4
G = -0.5 * D

# Convenience variables
dt2 = dt / 2
dt32 = 3 * dt / 2
A = ones(Nx) + dt2 * L
B = (ones(Nx) - dt2 * L) .^ -1
Nn = G .* fft(u0 .* u0)
Nn1 = Nn
iter = 2

# Initial time stepping
for n = 0:Nt-1
    Nn1 = Nn
    Nn = G .* fft(real(ifft(u)) .^ 2)
    u = B .* (A .* u + dt32 * Nn - dt2 * Nn1)

    if mod(n, nplot) == 0
        U[iter, :] = real(ifft(u))
        iter += 1
    end
end
hm = heatmap!(t, x, U; inspectable=false,
    colorrange=(-maximum(U), maximum(U)))
display(fig)

# Live time stepping and plotting
while events(fig).window_open[]
    for n = 0:nplot-1
        Nn1 = Nn
        Nn = G .* fft(real(ifft(u)) .^ 2)
        u = B .* (A .* u + dt32 * Nn - dt2 * Nn1)

        if mod(n, nplot) == 0  # Push through new data
            U = vcat(U[begin+1:end, :], real(ifft(u))')
            t = vcat(t[begin+1:end], [iter * dt * nplot])
            iter += 1
        end
    end
    # hm[1] = t # Why does this not work?
    hm[3] = U
    sleep(1 / fps)
end

### Audio in Julia
The most elementary audio module we'll use is `PortAudio`. It manages the annoying audio backend stuff for us, so that we can directly compute an audio buffer and write it to the sound card. The standard ouput stream can be set up via `stream = PortAudioStream(0, 2; samplerate=srate)` and closed with `close(stream)`. What we call the "audio buffer" is actually just an array where we play `srate` elements per second; so for the standard setup with an sample rate of `srate=44100` samples per second, the stream plays 44100 audio buffer array elements per second. To write the buffer (meaning, "playing" the audio buffer) to the sound card, we use `write(stream, buf)`.

To make the handling of audio data more convenient, we also use `SampledSignals` which provides useful types and function, and is compatible with `PortAudio`.

In [ ]:
using PortAudio, SampledSignals
srate  = 44100
stream = PortAudioStream(0, 2; samplerate=srate)

In [ ]:
# Create a simple sine wave audio buffer
duration = 2s
freq = 440

sbuf = SampleBuf(Float32, srate, duration, 2)
sbuf[:, 1] .= sin.(2pi * domain(sbuf) * freq)
sbuf[:, 2] .= sin.(1.02 * 2pi * domain(sbuf) * freq)
write(stream, sbuf)

f, ax, l1 = lines(domain(sbuf), sbuf[:, 1]; color=:green, axis=(xlabel="time [s]",
                  ylabel="amplitude", limits=(0, 2 / freq, -1.0, 1.0)))
lines!(ax, domain(sbuf), sbuf[:, 2]; color=:red)
f

Next, we want to vary (continuously or discretly) the sine frequency without getting audio clicks and pops. One way to achieve this is by varying the actual frequency in the sine function with each sample. This requires an a `freqsteps` vector which has the same length as the audio buffer, and in the easiest case, just ranges linearly from some start to a stop-frequency. This is computationally more expensive and also requires allocation of an extra array.

In [ ]:
# create frequency ramp audio buffer without clicks
duration = 3s
startfreq, stopfreq = 200, 1000

sbuf = SampleBuf(Float32, srate, duration, 2)
freqsteps = range(startfreq, stopfreq, nframes(sbuf))
sbuf[:, 1] .= 0.5sin.(2pi * domain(sbuf) * startfreq)
sbuf[:, 2] .= 0.5sin.(2pi * domain(sbuf) .* freqsteps)
write(stream, sbuf)

f, ax, l1 = lines(domain(sbuf), sbuf[:, 1]; label="ramp",
                  figure=(; resolution=(600, 250)),
                  axis=(; xlabel="time [s]", ylabel="amplitude"))
xlims!(0.0, 20 / startfreq)
lines!(ax, domain(sbuf), sbuf[:, 2]; linestyle=:dash, label="static")
axislegend(); f

Another way — the cheaper way — to avoid clicks when changing pitch is by providing the phase information between sine waves with different frequencies, to enable a sine with higher frequency to start playing right from the phase at which the sine with lower frequency stopped playing.

In [ ]:
# Create sample-and-hold-type buffer with random sine frequencies
duration  = 3s
sahrate   = 4Hz
randfreqs = 200 .+ 500 * rand((duration * sahrate).val)
lastphase = 0.0

sbuf = SampleBuf(Float32, srate, 1 / sahrate)
dom  = domain(sbuf)
for ν in randfreqs
    print(domain(sbuf))
    sbuf .= @. sin(lastphase + 2pi * dom * ν)
    lastphase = (lastphase + 2pi * (ν)Hz / sahrate) % 2pi
    write(stream, sbuf)
end

To convert the actual `U[t,x]` data to sound, we could take the Fourier spectrum of the space domain at some time step to obtain relative sine amplitudes from the Fourier coefficients. The array indices will be mapped onto actual audio frequencies in the 20Hz-20kHz domain. (Of course this mapping can be varied for interesting results.) One practical way to do this is by padding out the Fourier spectrum `fft(U[t,:])` with zeros such that the length of the array corresponds to `duration*srate`, where `duration` is the small time duration for which the considered time step is played. Then an inverse FFT is applied to obtain a real-numbered audio buffer.

In [ ]:
# Create test Fourier spectrum and transform to audio buffer
duration = 1
freqidx  = 400
u = zeros(Float64, duration * srate)
u[freqidx] = 1.0 * length(u)
unoise = randn(length(u))

sbuf = SampleBuf(real(ifft(u) .+ 20.0 * ifft(unoise)), srate)
write(stream, sbuf)

f = lines(domain(sbuf), sbuf.data; figure=(; resolution=(600, 250)),
          axis=(;xlabel="time [s]", ylabel="amplitude"))
xlims!(0.0, 2 / freq_idx); f

In [ ]:
# Translate U[t,x] cross section to audio buffer via FFT
# KS parameters
Lx = 128
Nx = 512
dt = 1 / 8
nplot = 4
Nt = 1200
slice = 40  # Cross section index

# Generate KS solution
x = Lx / Nx * collect(1:Nx)
u0 = @. cos(x) + 0.1 * cos(x / 16) * (1 + 2 * sin(x / 16))
U, t, x = ksIntegrateFast(u0, Lx, dt, Nt, nplot)
Unorm = U ./ maximum(abs.(U))  # Normalize solution to avoid clipping

# Audio buffer 
duration = 1s
sbuf = SampleBuf(Float32, srate, duration)
startfreq = 100
spectrum = zeros(Float32, length(sbuf.data))
spectrum[startfreq:startfreq+Nx-1] .= Unorm[slice, :]
# spectrum = vcat(zeros(Float64, startfreq - 1), Unorm[slice, :],
                # zeros(Float64, nframes(sbuf) - (Nx + startfreq - 1)));
sbuf .= real(ifft(spectrum))
sbuf .= sbuf.data / maximum(abs.(sbuf.data))
write(stream, sbuf)

f = Figure(; resolution=(800, 250))
axleft  = Axis(f[1, 1]; xlabel="time [s]", ylabel="amplitude")
axright = Axis(f[1, 2]; xlabel="frequency [Hz]", xscale=log10)
lines!(axleft, domain(sbuf), sbuf.data)
lines!(axright, 1:nframes(sbuf), spectrum)
xlims!(axright, 20, 10000); f

The previous way to generate audio produces generally transient sounds. What would be more (artistically) useful is a translation of the frequency spectrum to constant sine stacks. Adjacent cross sections `U[t,:]` would then continously "morph" into each other.

So what we need is a function that takes an input spectrum and returns a sine stack where the amplitudes of different frequencies are attenuated according to `U[t,x]` (and maybe some extra mapping).

In [ ]:
# Translate U[t,:] to audio buffer via sine stack
function sinestack!(sbuf::SampleBuf, u::Vector, freqs::Vector, ampmap, lastphases::Vector)
    dom = domain(sbuf)
    for (i, ν) in enumerate(freqs)
        sbuf.data .+= ampmap(u[i]) .* sin.(lastphases[i] .+ 2pi * dom * ν)
        lastphases[i] = (lastphases[i] + 2pi * dom[end] * ν) % 2pi
    end
    sbuf .= sbuf.data / maximum(sbuf.data)
    lastphases
end

In [ ]:
# KS parameters
Lx = 32
Nx = 64
dt = 1 / 16
nplot = 4
Nt = 1200
slice = 60

x  = Lx / Nx * collect(1:Nx)
u0 = @. cos(x) + 0.1 * cos(x / 16) * (1 + 2 * sin(x / 16))
U, t, x = ksIntegrateFast(u0, Lx, dt, Nt, nplot)
Unorm = U ./ maximum(abs.(U))

duration = 2s
freqs = collect(range(200, 400, Nx))
am = x -> x^6
sbuf = SampleBuf(Float32, srate, duration)
sbuf .= 0.0
lastphases = sinestack!(sbuf, Unorm[slice, :], freqs, am, zeros(Float64, Nx))
write(stream, sbuf);

In [ ]:
# Finalize by closing stream
close(stream);